In [295]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from google_play_scraper import Sort, reviews_all, reviews
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
from deep_translator import GoogleTranslator
import emoji
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from googletrans import Translator
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [2]:
pd.set_option('display.max_colwidth', 100)

In [2]:
#ingest all the newest reviews in google play
results = reviews_all(
    'com.tencent.qqlivei18n',
    sleep_milliseconds=0, # defaults to 0
    lang='id', # defaults to 'en'
    country='id', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

#create dataframe
df = pd.DataFrame(np.array(results),columns=['review'])
df = df.join(pd.DataFrame(df.pop('review').tolist()))
df = df[['reviewId','content','score','reviewCreatedVersion','at']] ##remove
df


,reviewId,content,score,reviewCreatedVersion,at
0,gp:AOqpTOHje5HS_WGRRqrbpqAoaZNs1J62PB8ABSrg5D4...,OK.......,5,None,2022-03-01 07:45:41
1,gp:AOqpTOF6SVtvOQJDqZDJaKlbjIYjN_gGdyujH_q9eQR...,Aplikasi ini sangat bagus,5,None,2022-03-01 06:57:36
2,gp:AOqpTOGR3hYx7_xmpHUf5M3AIjt9k9sgTIeeE1OEIh7...,gua udh berhenti berlangganan tapi malah tetep...,1,None,2022-03-01 06:05:30
3,gp:AOqpTOGNW1Xxtbn2iVF9mo8NSSMiqCvcPLi6fL27ZdB...,"Gua donlot ni gara2 soul land,. Aaaaarght terl...",5,4.8.9.7980,2022-03-01 05:36:38
4,gp:AOqpTOF6yQc3U17No2ybVtrlTrgQlCIat31TEZYgy7h...,SAYA KECEWA SAMA APLIKASI INI. SAYA SUDAH BELI...,1,None,2022-03-01 03:37:11
...,...,...,...,...,...
72240,gp:AOqpTOEdvSYqEIkRlgi2dzdo4dyi0-ZexXgmdGDer3J...,mantap,5,None,2019-07-11 23:10:57
72241,gp:AOqpTOHDctQaHJONgw9xc8KPqJWbo2nVzNcC2aviF0s...,Aku senang sekali dengan adanya ini aku bisa m...,4,1.7.5.5114,2019-07-10 23:28:23
72242,gp:AOqpTOHI_hQcW4QwjCwcfvGUdSAZDXso-PyRcz54vvI...,good,5,None,2019-07-10 10:54:01
72243,gp:AOqpTOFZKR59WD04gUgg7V7iS-MgIxktC4DhuG2xqh5...,I ask for an Indonesian translation and I will...,1,1.5.8.5063,2019-05-10 08:20:57


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72245 entries, 0 to 72244
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              72245 non-null  object        
 1   content               72245 non-null  object        
 2   score                 72245 non-null  int64         
 3   reviewCreatedVersion  41580 non-null  object        
 4   at                    72245 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 2.8+ MB


In [18]:
df['reviewId'] = df['reviewId'].str[3:]
df

,reviewId,content,score,reviewCreatedVersion,at
0,AOqpTOHje5HS_WGRRqrbpqAoaZNs1J62PB8ABSrg5D4T1O...,OK.......,5,None,2022-03-01 07:45:41
1,AOqpTOF6SVtvOQJDqZDJaKlbjIYjN_gGdyujH_q9eQRw9Z...,Aplikasi ini sangat bagus,5,None,2022-03-01 06:57:36
2,AOqpTOGR3hYx7_xmpHUf5M3AIjt9k9sgTIeeE1OEIh7mCt...,gua udh berhenti berlangganan tapi malah tetep...,1,None,2022-03-01 06:05:30
3,AOqpTOGNW1Xxtbn2iVF9mo8NSSMiqCvcPLi6fL27ZdB7Vw...,"Gua donlot ni gara2 soul land,. Aaaaarght terl...",5,4.8.9.7980,2022-03-01 05:36:38
4,AOqpTOF6yQc3U17No2ybVtrlTrgQlCIat31TEZYgy7hPbZ...,SAYA KECEWA SAMA APLIKASI INI. SAYA SUDAH BELI...,1,None,2022-03-01 03:37:11
...,...,...,...,...,...
72240,AOqpTOEdvSYqEIkRlgi2dzdo4dyi0-ZexXgmdGDer3JcS5...,mantap,5,None,2019-07-11 23:10:57
72241,AOqpTOHDctQaHJONgw9xc8KPqJWbo2nVzNcC2aviF0sBHc...,Aku senang sekali dengan adanya ini aku bisa m...,4,1.7.5.5114,2019-07-10 23:28:23
72242,AOqpTOHI_hQcW4QwjCwcfvGUdSAZDXso-PyRcz54vvIHfJ...,good,5,None,2019-07-10 10:54:01
72243,AOqpTOFZKR59WD04gUgg7V7iS-MgIxktC4DhuG2xqh5fy6...,I ask for an Indonesian translation and I will...,1,1.5.8.5063,2019-05-10 08:20:57


In [19]:
df_jan = df.loc[(df['at'] > '2022-01-01') & (df['at'] < '2022-01-31')]
df_jan

,reviewId,content,score,reviewCreatedVersion,at
1815,AOqpTOFtZoGEc2AH2RjQO29b9zWwbZcAfQtscflV9pcpyu...,Gimana cara berhenti berlangganan ya ???,3,None,2022-01-30 23:55:38
1816,AOqpTOHoFTJvcf-kiA96MxIa5CMXqht7jdEJPC0eI-4CDc...,Bagus😊,5,None,2022-01-30 23:46:36
1817,AOqpTOGd1-GCpDHBPwCU-PiXhvXMewV3HLlozBdvAxyD5c...,Kalau bisa iklannya ya wak jangan lemot,5,None,2022-01-30 23:44:37
1818,AOqpTOHNCWlku6s5hK1ORa7bV5DeuYvQlcCOB48Iq2zhEq...,Good 👍🏻👍🏻,5,None,2022-01-30 23:44:15
1819,AOqpTOGPRnZhaDFWRvNDBt57Y4usBDPbLK5gjI8DUPD6Ua...,"susah sekalii download nya, udah 100% eh ulang...",1,None,2022-01-30 22:44:29
...,...,...,...,...,...
4409,AOqpTOEYOYef3dzvRx0wKQQ1kkEAsl-p1ZzE0Y2y3HZG7K...,Sukses trus,2,None,2022-01-01 01:36:43
4410,AOqpTOHamtQY6_oD6_jPmLcYLw_wb3wFZdmqmCd5kpBgWH...,Payah pke beli,1,4.7.6.7610,2022-01-01 01:28:16
4411,AOqpTOGtqPVNYNLtjx9Jq8F4Elr2k4w4CJSAosli-a99Lt...,Mantaappp,5,4.7.6.7610,2022-01-01 00:59:18
4412,AOqpTOHr0wdsyDHxT2TNTDQfWbW6cpnOI4Lb-nYcWNpuqu...,Yoi men,5,None,2022-01-01 00:44:47


In [201]:
df_jan.nunique(axis=0)

reviewId                2599
content                 2112
score                      5
reviewCreatedVersion      56
at                      2596
dtype: int64

In [ ]:
#transform column's name

#schema data type
#review_id -> string
#date -> datetime
#app_name -> string
#content -> string
#app_version -> string
#gp_score -> int
#sentiment_score -> boolean (positive and negative)

#extract data with column comply schema and date in range
#date -> 1 minggu

#load? bigquery??

In [183]:
df_sentiment = df_jan[['reviewId','at','content']]


,reviewId,at,content
1815,AOqpTOFtZoGEc2AH2RjQO29b9zWwbZcAfQtscflV9pcpyu...,2022-01-30 23:55:38,Gimana cara berhenti berlangganan ya ???
1816,AOqpTOHoFTJvcf-kiA96MxIa5CMXqht7jdEJPC0eI-4CDc...,2022-01-30 23:46:36,Bagus😊
1817,AOqpTOGd1-GCpDHBPwCU-PiXhvXMewV3HLlozBdvAxyD5c...,2022-01-30 23:44:37,Kalau bisa iklannya ya wak jangan lemot
1818,AOqpTOHNCWlku6s5hK1ORa7bV5DeuYvQlcCOB48Iq2zhEq...,2022-01-30 23:44:15,Good 👍🏻👍🏻
1819,AOqpTOGPRnZhaDFWRvNDBt57Y4usBDPbLK5gjI8DUPD6Ua...,2022-01-30 22:44:29,"susah sekalii download nya, udah 100% eh ulang..."
...,...,...,...
4409,AOqpTOEYOYef3dzvRx0wKQQ1kkEAsl-p1ZzE0Y2y3HZG7K...,2022-01-01 01:36:43,Sukses trus
4410,AOqpTOHamtQY6_oD6_jPmLcYLw_wb3wFZdmqmCd5kpBgWH...,2022-01-01 01:28:16,Payah pke beli
4411,AOqpTOGtqPVNYNLtjx9Jq8F4Elr2k4w4CJSAosli-a99Lt...,2022-01-01 00:59:18,Mantaappp
4412,AOqpTOHr0wdsyDHxT2TNTDQfWbW6cpnOI4Lb-nYcWNpuqu...,2022-01-01 00:44:47,Yoi men


In [344]:
def remove_special_char(text):
    return re.sub(r"[^a-zA-Z0-9\s]","", text)

def strip_emoji(text):
    new_text = re.sub(emoji.get_emoji_regexp(), r"", text)
    return new_text

def remove_multiple_char(text):
    text_token = word_tokenize(text)

    words = []
    for word in text_token:
        if re.search(r"(.)\1{2,}", word):
            new_word = re.split(r"(.)\1{2,}", word)
            new_word = ''.join(new_word)
        else:
            new_word = word
        words.append(new_word)
    new_text = ' '.join(words)
    return new_text

def translator(text):
  new_text = GoogleTranslator(source='id', target='en').translate(text)
  return new_text

def sentiment_score(text):
    testimonial = TextBlob(text)
    score = testimonial.sentiment[0]
    return score

In [187]:
#case folding
df_sentiment['content'] = df_sentiment['content'].str.lower()

#remove special character
df_sentiment['content'] = df_sentiment['content'].apply(remove_special_char)

#remove emoji
df_sentiment['content'] = df_sentiment['content'].apply(strip_emoji)

#remove multiple character
df_sentiment['content'] = df_sentiment['content'].apply(remove_multiple_char)

#remove blank row
df_sentiment = df_sentiment.replace(r'^\s*$', float("NaN"), regex=True)
df_sentiment = df_sentiment.replace(r'^([0-9]*)$', float("NaN"), regex=True)
df_sentiment.dropna(inplace=True)

#translate id to en
df_sentiment['content_en'] = df_sentiment['content'].apply(translator)

#calculate sentiment score
df_sentiment['score'] = df_sentiment['content_en'].apply(sentiment_score)

#map sentiment label
df_sentiment['label'] = pd.cut(x=df_sentiment['score'], bins=[-1,-0.000009,0.000009,1],
                     labels=['negative', 'neutral','positive'])


/var/folders/9b/3tbz57ps2vj36lwt_1gsjkdh0000gn/T/ipykernel_6156/1652761030.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['content'] = df_sentiment['content'].str.lower()
/var/folders/9b/3tbz57ps2vj36lwt_1gsjkdh0000gn/T/ipykernel_6156/1652761030.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['content'] = df_sentiment['content'].apply(remove_special_char)
/var/folders/9b/3tbz57ps2vj36lwt_1gsjkdh0000gn/T/ipykernel_6156/1652761030.py:8: SettingWithCopyWarning: 
A value is tr

,reviewId,at,content
1815,AOqpTOFtZoGEc2AH2RjQO29b9zWwbZcAfQtscflV9pcpyu...,2022-01-30 23:55:38,gimana cara berhenti berlangganan ya
1816,AOqpTOHoFTJvcf-kiA96MxIa5CMXqht7jdEJPC0eI-4CDc...,2022-01-30 23:46:36,bagus
1817,AOqpTOGd1-GCpDHBPwCU-PiXhvXMewV3HLlozBdvAxyD5c...,2022-01-30 23:44:37,kalau bisa iklannya ya wak jangan lemot
1818,AOqpTOHNCWlku6s5hK1ORa7bV5DeuYvQlcCOB48Iq2zhEq...,2022-01-30 23:44:15,good
1819,AOqpTOGPRnZhaDFWRvNDBt57Y4usBDPbLK5gjI8DUPD6Ua...,2022-01-30 22:44:29,susah sekalii download nya udah 100 eh ulang l...
...,...,...,...
4409,AOqpTOEYOYef3dzvRx0wKQQ1kkEAsl-p1ZzE0Y2y3HZG7K...,2022-01-01 01:36:43,sukses trus
4410,AOqpTOHamtQY6_oD6_jPmLcYLw_wb3wFZdmqmCd5kpBgWH...,2022-01-01 01:28:16,payah pke beli
4411,AOqpTOGtqPVNYNLtjx9Jq8F4Elr2k4w4CJSAosli-a99Lt...,2022-01-01 00:59:18,mantaappp
4412,AOqpTOHr0wdsyDHxT2TNTDQfWbW6cpnOI4Lb-nYcWNpuqu...,2022-01-01 00:44:47,yoi men


In [320]:
df_negative = df_sentiment[df_sentiment['label']=='negative']

In [321]:
def translator_en_id(text):
  result = GoogleTranslator(source='en', target='id').translate(text)
  #TextBlob(text).translate(from_lang='en', to="id")
  #translator = Translator()
  #result = translator.translate(text, src='en', dest='id')
  return result

In [322]:
#df_negative['content_id'] = df_negative['content_en'].apply(translator_en_id)
#df_negative

In [323]:
def remove_number(string):
    return re.sub(r"[0-9]","",string)

In [324]:
df_negative['content_en'] = df_negative['content_en'].str.lower()

#remove special character
df_negative['content_en'] = df_negative['content_en'].apply(remove_special_char)

#remove_number
df_negative['content_en'] = df_negative['content_en'].apply(remove_number)

#remove emoji
df_negative['content_en'] = df_negative['content_en'].apply(strip_emoji)

#remove blank row
df_negative = df_negative.replace(r'^\s*$', float("NaN"), regex=True)
df_negative = df_negative.replace(r'^([0-9]*)$', float("NaN"), regex=True)
df_negative.dropna(inplace=True)

/var/folders/9b/3tbz57ps2vj36lwt_1gsjkdh0000gn/T/ipykernel_6156/441305470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_negative['content_en'] = df_negative['content_en'].str.lower()
/var/folders/9b/3tbz57ps2vj36lwt_1gsjkdh0000gn/T/ipykernel_6156/441305470.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_negative['content_en'] = df_negative['content_en'].apply(remove_special_char)
/var/folders/9b/3tbz57ps2vj36lwt_1gsjkdh0000gn/T/ipykernel_6156/441305470.py:7: SettingWithCopyWarning: 
A value 

In [325]:
df_negative['token'] = df_negative['content_en'].apply(word_tokenize)

In [326]:
df_negative

,reviewId,at,content,content_en,score,label,token
1817,AOqpTOGd1-GCpDHBPwCU-PiXhvXMewV3HLlozBdvAxyD5c...,2022-01-30 23:44:37,kalau bisa iklannya ya wak jangan lemot,if you can advertise bro dont be slow,-0.300000,negative,"[if, you, can, advertise, bro, dont, be, slow]"
1819,AOqpTOGPRnZhaDFWRvNDBt57Y4usBDPbLK5gjI8DUPD6Ua...,2022-01-30 22:44:29,susah sekalii download nya udah 100 eh ulang l...,its really hard to download its already eh it...,-0.291667,negative,"[its, really, hard, to, download, its, already..."
1828,AOqpTOEJWedKwvIgsXaeqVhLF6jR-QVtcezY_IC5pbCGxx...,2022-01-30 20:43:10,sebelumnya bisa banyak nonton drakor tanpa vip...,previously you could watch a lot of drakor wit...,-0.166667,negative,"[previously, you, could, watch, a, lot, of, dr..."
1832,AOqpTOHSE2_7JK_119JxKPecYFcybFnQOLsIYKl5edrom2...,2022-01-30 20:21:32,saldo udh ada mau bayar aja ribet aksesnya,the balance already exists want to pay its dif...,-0.500000,negative,"[the, balance, already, exists, want, to, pay,..."
1850,AOqpTOHaQLyeMQwGQViLS9MtNVwO07uWYnD0DVC-pzlSHv...,2022-01-30 15:56:11,jelek aplikasinya byk iklan sm srg nge lag,the application is bad there are a lot of ads ...,-0.700000,negative,"[the, application, is, bad, there, are, a, lot..."
...,...,...,...,...,...,...,...
4391,AOqpTOEpy1W0zWs06pIZe3Uud4Wl0IkWooUf3q06r8ylr1...,2022-01-01 08:43:13,minim,minimal,-0.100000,negative,[minimal]
4400,AOqpTOHBHN3FN-V7NhNXx1lf_41VmwKsfUuvDUEkP0d0Hd...,2022-01-01 03:27:55,gimana ko ga bs2 daftar pake shopepay ga bs pa...,how come you cant register using shopepay cant...,-0.650000,negative,"[how, come, you, cant, register, using, shopep..."
4404,AOqpTOF7mIHG13qP13mhW5QIbMYKcDdc-iRO2givutlQwt...,2022-01-01 02:16:24,iklankonten sangat tidak layak untuk di lihat ...,content ads are not worth seeing because they ...,-0.150000,negative,"[content, ads, are, not, worth, seeing, becaus..."
4405,AOqpTOGwmPHpaesfqaJuzji-SFd4U-U69cXKy3oB8-Yi7M...,2022-01-01 02:14:29,chanel animenya kurang banyak,there are not many anime channels,-0.250000,negative,"[there, are, not, many, anime, channels]"


In [327]:
def stemming(token):
    ps = PorterStemmer()
    words = []
    for word in token:
        word = ps.stem(word)
        words.append(word)
    return words

In [328]:
# def stemming(token):
#     stemmer = StemmerFactory().create_stemmer()
#     words = []
#     for word in token:
#         word = stemmer.stem(str(word))
#         words.append(word)
#     return words

In [329]:
df_negative['stemming'] = df_negative['token'].apply(stemming)

In [330]:
df_negative

,reviewId,at,content,content_en,score,label,token,stemming
1817,AOqpTOGd1-GCpDHBPwCU-PiXhvXMewV3HLlozBdvAxyD5c...,2022-01-30 23:44:37,kalau bisa iklannya ya wak jangan lemot,if you can advertise bro dont be slow,-0.300000,negative,"[if, you, can, advertise, bro, dont, be, slow]","[if, you, can, advertis, bro, dont, be, slow]"
1819,AOqpTOGPRnZhaDFWRvNDBt57Y4usBDPbLK5gjI8DUPD6Ua...,2022-01-30 22:44:29,susah sekalii download nya udah 100 eh ulang l...,its really hard to download its already eh it...,-0.291667,negative,"[its, really, hard, to, download, its, already...","[it, realli, hard, to, download, it, alreadi, ..."
1828,AOqpTOEJWedKwvIgsXaeqVhLF6jR-QVtcezY_IC5pbCGxx...,2022-01-30 20:43:10,sebelumnya bisa banyak nonton drakor tanpa vip...,previously you could watch a lot of drakor wit...,-0.166667,negative,"[previously, you, could, watch, a, lot, of, dr...","[previous, you, could, watch, a, lot, of, drak..."
1832,AOqpTOHSE2_7JK_119JxKPecYFcybFnQOLsIYKl5edrom2...,2022-01-30 20:21:32,saldo udh ada mau bayar aja ribet aksesnya,the balance already exists want to pay its dif...,-0.500000,negative,"[the, balance, already, exists, want, to, pay,...","[the, balanc, alreadi, exist, want, to, pay, i..."
1850,AOqpTOHaQLyeMQwGQViLS9MtNVwO07uWYnD0DVC-pzlSHv...,2022-01-30 15:56:11,jelek aplikasinya byk iklan sm srg nge lag,the application is bad there are a lot of ads ...,-0.700000,negative,"[the, application, is, bad, there, are, a, lot...","[the, applic, is, bad, there, are, a, lot, of,..."
...,...,...,...,...,...,...,...,...
4391,AOqpTOEpy1W0zWs06pIZe3Uud4Wl0IkWooUf3q06r8ylr1...,2022-01-01 08:43:13,minim,minimal,-0.100000,negative,[minimal],[minim]
4400,AOqpTOHBHN3FN-V7NhNXx1lf_41VmwKsfUuvDUEkP0d0Hd...,2022-01-01 03:27:55,gimana ko ga bs2 daftar pake shopepay ga bs pa...,how come you cant register using shopepay cant...,-0.650000,negative,"[how, come, you, cant, register, using, shopep...","[how, come, you, cant, regist, use, shopepay, ..."
4404,AOqpTOF7mIHG13qP13mhW5QIbMYKcDdc-iRO2givutlQwt...,2022-01-01 02:16:24,iklankonten sangat tidak layak untuk di lihat ...,content ads are not worth seeing because they ...,-0.150000,negative,"[content, ads, are, not, worth, seeing, becaus...","[content, ad, are, not, worth, see, becaus, th..."
4405,AOqpTOGwmPHpaesfqaJuzji-SFd4U-U69cXKy3oB8-Yi7M...,2022-01-01 02:14:29,chanel animenya kurang banyak,there are not many anime channels,-0.250000,negative,"[there, are, not, many, anime, channels]","[there, are, not, mani, anim, channel]"


In [331]:
def filtering(stemming):
    wordlist    = set(stopwords.words('english'))
    result   = [i for i in stemming if i not in wordlist]
    return result


In [332]:
df_negative['filtering'] = df_negative['stemming'].apply(filtering)

In [333]:
words = []
for list in df_negative['filtering']:
    for word in list:
        words.append(word)
words

['advertis',
 'bro',
 'dont',
 'slow',
 'realli',
 'hard',
 'download',
 'alreadi',
 'eh',
 'doesnt',
 'instal',
 'hmm',
 'previous',
 'could',
 'watch',
 'lot',
 'drakor',
 'without',
 'vip',
 'whi',
 'ha',
 'vip',
 'balanc',
 'alreadi',
 'exist',
 'want',
 'pay',
 'difficult',
 'access',
 'applic',
 'bad',
 'lot',
 'ad',
 'sm',
 'often',
 'lag',
 'use',
 'thi',
 'applic',
 'come',
 'applic',
 'come',
 'suddenli',
 'im',
 'annoy',
 'pleas',
 'fix',
 'im',
 'comfort',
 'im',
 'give',
 'half',
 'star',
 'becaus',
 'im',
 'ilok',
 'network',
 'havent',
 'tri',
 'yet',
 'satisfi',
 'becaus',
 'drama',
 'want',
 'see',
 'thi',
 'applic',
 'apk',
 'friendli',
 'lag',
 'clear',
 'doesnt',
 'seem',
 'like',
 'old',
 'load',
 'forzi',
 'network',
 'fix',
 'emot',
 'take',
 'long',
 'download',
 'sound',
 'wetv',
 'whi',
 'want',
 'enter',
 'vip',
 'realli',
 'difficult',
 'becaus',
 'credit',
 'enough',
 'even',
 'though',
 'credit',
 'applic',
 'give',
 'wast',
 'balanc',
 'pull',
 'continu',

In [334]:
translated = GoogleTranslator('en', 'id').translate_batch(words)
translated

['iklan',
 'kawan',
 'jangan',
 'lambat',
 'benar-benar',
 'keras',
 'unduh',
 'sudah',
 'eh',
 'tidak',
 'pasang',
 'Hmm',
 'sebelumnya',
 'bisa',
 'jam tangan',
 'banyak',
 'drakor',
 'tanpa',
 'vip',
 'yang',
 'Ha',
 'vip',
 'keseimbangan',
 'sudah',
 'ada',
 'mau',
 'membayar',
 'sulit',
 'mengakses',
 'aplikasi',
 'buruk',
 'banyak',
 'iklan',
 'sm',
 'sering',
 'ketinggalan',
 'menggunakan',
 'ini',
 'aplikasi',
 'datang',
 'aplikasi',
 'datang',
 'tiba-tiba',
 'saya',
 'mengganggu',
 'mohon',
 'memperbaiki',
 'saya',
 'kenyamanan',
 'saya',
 'memberi',
 'setengah',
 'bintang',
 'karena',
 'saya',
 'ilok',
 'jaringan',
 'belum',
 'tiga',
 'belum',
 'memuaskan',
 'karena',
 'drama',
 'mau',
 'melihat',
 'ini',
 'aplikasi',
 'apk',
 'persahabatan',
 'ketinggalan',
 'jernih',
 'tidak',
 'terlihat',
 'Suka',
 'tua',
 'memuat',
 'forzi',
 'jaringan',
 'memperbaiki',
 'emosi',
 'mengambil',
 'panjang',
 'unduh',
 'suara',
 'wetv',
 'yang',
 'mau',
 'memasuki',
 'vip',
 'benar-benar',
 

In [335]:
len(translated)

4717

In [339]:
def filtering_id(stemming):
    wordlist    = set(stopwords.words('indonesia'))
    result   = [i for i in stemming if i not in wordlist]
    return result

In [340]:
translated_filter = filtering_id(translated)
translated_filter

['iklan',
 'kawan',
 'lambat',
 'benar-benar',
 'keras',
 'unduh',
 'eh',
 'pasang',
 'Hmm',
 'jam tangan',
 'drakor',
 'vip',
 'Ha',
 'vip',
 'keseimbangan',
 'membayar',
 'sulit',
 'mengakses',
 'aplikasi',
 'buruk',
 'iklan',
 'sm',
 'ketinggalan',
 'aplikasi',
 'aplikasi',
 'mengganggu',
 'mohon',
 'memperbaiki',
 'kenyamanan',
 'bintang',
 'ilok',
 'jaringan',
 'memuaskan',
 'drama',
 'aplikasi',
 'apk',
 'persahabatan',
 'ketinggalan',
 'jernih',
 'Suka',
 'tua',
 'memuat',
 'forzi',
 'jaringan',
 'memperbaiki',
 'emosi',
 'mengambil',
 'unduh',
 'suara',
 'wetv',
 'memasuki',
 'vip',
 'benar-benar',
 'sulit',
 'kredit',
 'kredit',
 'aplikasi',
 'limbah',
 'keseimbangan',
 'menarik',
 'lanjutkan',
 'berlangganan',
 'Ha',
 'berhenti',
 'aktif',
 'mengasapi',
 'pasang',
 'pasang',
 'film',
 'veri',
 'subjudul',
 'hilang',
 'wetv',
 'server',
 'kuat',
 'mpb',
 'jaringan',
 'terjebak',
 'kehilangan',
 'membeli',
 'vip',
 'improvisasi',
 'kualitas',
 'server',
 'gagal',
 'unduh',
 'tr

In [341]:
len(translated_filter)

3403

In [342]:
from nltk import ngrams, FreqDist

fdist1 = FreqDist(translated)
fdist1
filtered_word_freq = dict((word, freq) for word, freq in fdist1.items() if not word.isdigit())
filtered_word_freq

{'iklan': 20,
 'kawan': 5,
 'jangan': 50,
 'lambat': 18,
 'benar-benar': 68,
 'keras': 26,
 'unduh': 68,
 'sudah': 37,
 'eh': 1,
 'tidak': 34,
 'pasang': 12,
 'Hmm': 2,
 'sebelumnya': 3,
 'bisa': 1,
 'jam tangan': 102,
 'banyak': 15,
 'drakor': 2,
 'tanpa': 4,
 'vip': 225,
 'yang': 68,
 'Ha': 47,
 'keseimbangan': 21,
 'ada': 2,
 'mau': 83,
 'membayar': 37,
 'sulit': 16,
 'mengakses': 4,
 'aplikasi': 88,
 'buruk': 67,
 'sm': 2,
 'sering': 12,
 'ketinggalan': 4,
 'menggunakan': 46,
 'ini': 79,
 'datang': 41,
 'tiba-tiba': 11,
 'saya': 132,
 'mengganggu': 35,
 'mohon': 52,
 'memperbaiki': 21,
 'kenyamanan': 3,
 'memberi': 17,
 'setengah': 2,
 'bintang': 13,
 'karena': 47,
 'ilok': 1,
 'jaringan': 9,
 'belum': 21,
 'tiga': 15,
 'memuaskan': 3,
 'drama': 13,
 'melihat': 12,
 'apk': 31,
 'persahabatan': 2,
 'jernih': 17,
 'terlihat': 2,
 'Suka': 43,
 'tua': 4,
 'memuat': 4,
 'forzi': 1,
 'emosi': 3,
 'mengambil': 9,
 'panjang': 13,
 'suara': 9,
 'wetv': 27,
 'memasuki': 19,
 'kredit': 40,
 '

In [343]:
sort_orders = sorted(filtered_word_freq.items(), key=lambda x: x[1], reverse=True)
sort_orders

[('vip', 225),
 ('tidak bisa', 134),
 ('saya', 132),
 ('jam tangan', 102),
 ('aplikasi', 88),
 ('berlangganan', 84),
 ('mau', 83),
 ('ini', 79),
 ('bahkan', 76),
 ('benar-benar', 68),
 ('unduh', 68),
 ('yang', 68),
 ('buruk', 67),
 ('meskipun', 56),
 ('mengecewakan', 54),
 ('mohon', 52),
 ('jangan', 50),
 ('Ha', 47),
 ('karena', 47),
 ('menggunakan', 46),
 ('Suka', 43),
 ('datang', 41),
 ('kredit', 40),
 ('sudah', 37),
 ('membayar', 37),
 ('wa', 36),
 ('mengganggu', 35),
 ('tidak', 34),
 ('maaf', 34),
 ('veri', 33),
 ('tetap', 33),
 ('film', 32),
 ('apk', 31),
 ('membeli', 31),
 ('dibayar', 30),
 ('pembayaran', 29),
 ('gagal', 28),
 ('wetv', 27),
 ('membuat', 27),
 ('keras', 26),
 ('waktu', 23),
 ('bulan', 23),
 ('daftar', 23),
 ('memperbarui', 22),
 ('bagus', 22),
 ('keseimbangan', 21),
 ('memperbaiki', 21),
 ('belum', 21),
 ('Akun', 21),
 ('uang', 21),
 ('iklan', 20),
 ('subjudul', 20),
 ('dibeli', 20),
 ('membuka', 20),
 ('memasuki', 19),
 ('memotong', 19),
 ('bermain', 19),
 ('epis

In [75]:
#1856

In [ ]:
#repeated words
# {word}2
# {word}"

In [ ]:
#non-standard abbreviation